# Challenge SRL Models - Main Script

In [3]:
import json
import re
import random

import checklist
from checklist.editor import Editor
from checklist.perturb import Perturb
from checklist.test_types import MFT, INV, DIR
from checklist.expect import Expect
from checklist.pred_wrapper import PredictorWrapper

In [106]:
%run utils.ipynb
%run generation_voice.ipynb
%run generation_long_distance.ipynb
%run generation_loc_vs_dir.ipynb
%run generation_dativ_alternation.ipynb

In [56]:
import warnings
warnings.filterwarnings('ignore')

# Initialize random seed

import torch
seed = 423
random.seed(seed)
torch.manual_seed(seed)

In [16]:
from allennlp_models.pretrained import load_predictor

In [17]:
srl_predictor = load_predictor('structured-prediction-srl') 

#srl_predictor = load_predictor('structured-prediction-srl-bert')

2022-03-31 13:07:59,493 - INFO - allennlp.common.plugins - Plugin allennlp_models available
2022-03-31 13:07:59,629 - INFO - allennlp.common.params - id = vgqa-vilbert
2022-03-31 13:07:59,631 - INFO - allennlp.common.params - registered_model_name = vqa_vilbert_from_huggingface
2022-03-31 13:07:59,632 - INFO - allennlp.common.params - model_class = None
2022-03-31 13:07:59,654 - INFO - allennlp.common.params - registered_predictor_name = vgqa_vilbert
2022-03-31 13:07:59,665 - INFO - allennlp.common.params - display_name = ViLBERT - Visual Genome Question Answering
2022-03-31 13:07:59,670 - INFO - allennlp.common.params - task_id = vgqa
2022-03-31 13:07:59,672 - INFO - allennlp.common.params - model_usage.archive_file = vilbert-vgqa-pretrained.2021-05-10.tar.gz
2022-03-31 13:07:59,677 - INFO - allennlp.common.params - model_usage.training_config = vision/vilbert_vgqa_pretrained.jsonnet
2022-03-31 13:07:59,678 - INFO - allennlp.common.params - model_usage.install_instructions = pip insta

2022-03-31 13:08:00,070 - INFO - allennlp.common.params - model_details.paper.citation = 
@article{Shi2019SimpleBM,
title={Simple BERT Models for Relation Extraction and Semantic Role Labeling},
author={Peng Shi and Jimmy Lin},
journal={ArXiv},
year={2019},
volume={abs/1904.05255}}

2022-03-31 13:08:00,073 - INFO - allennlp.common.params - model_details.paper.title = Simple BERT Models for Relation Extraction and Semantic Role Labeling
2022-03-31 13:08:00,075 - INFO - allennlp.common.params - model_details.paper.url = https://api.semanticscholar.org/CorpusID:131773936
2022-03-31 13:08:00,077 - INFO - allennlp.common.params - model_details.license = None
2022-03-31 13:08:00,082 - INFO - allennlp.common.params - model_details.contact = allennlp-contact@allenai.org
2022-03-31 13:08:00,088 - INFO - allennlp.common.params - intended_use.primary_uses = None
2022-03-31 13:08:00,100 - INFO - allennlp.common.params - intended_use.primary_users = None
2022-03-31 13:08:00,102 - INFO - allennlp.co

2022-03-31 13:08:00,786 - INFO - allennlp.common.params - model_class = None
2022-03-31 13:08:00,787 - INFO - allennlp.common.params - registered_predictor_name = None
2022-03-31 13:08:00,787 - INFO - allennlp.common.params - display_name = Deep Biaffine Attention for Neural Dependency Parsing
2022-03-31 13:08:00,792 - INFO - allennlp.common.params - task_id = dependency-parsing
2022-03-31 13:08:00,796 - INFO - allennlp.common.params - model_usage.archive_file = biaffine-dependency-parser-ptb-2020.04.06.tar.gz
2022-03-31 13:08:00,798 - INFO - allennlp.common.params - model_usage.training_config = structured_prediction/dependency_parser.jsonnet
2022-03-31 13:08:00,799 - INFO - allennlp.common.params - model_usage.install_instructions = pip install allennlp==2.1.0 allennlp-models==2.1.0
2022-03-31 13:08:00,803 - INFO - allennlp.common.params - model_usage.overrides = None
2022-03-31 13:08:00,810 - INFO - allennlp.common.params - model_details.description = This dependency parser follows 

2022-03-31 13:08:01,128 - INFO - allennlp.common.params - model_details.paper.title = RoBERTa: A Robustly Optimized BERT Pretraining Approach (Liu et al)
2022-03-31 13:08:01,132 - INFO - allennlp.common.params - model_details.paper.url = https://api.semanticscholar.org/CorpusID:198953378
2022-03-31 13:08:01,134 - INFO - allennlp.common.params - model_details.license = None
2022-03-31 13:08:01,145 - INFO - allennlp.common.params - model_details.contact = allennlp-contact@allenai.org
2022-03-31 13:08:01,153 - INFO - allennlp.common.params - intended_use.primary_uses = None
2022-03-31 13:08:01,160 - INFO - allennlp.common.params - intended_use.primary_users = None
2022-03-31 13:08:01,161 - INFO - allennlp.common.params - intended_use.out_of_scope_use_cases = None
2022-03-31 13:08:01,163 - INFO - allennlp.common.params - factors.relevant_factors = None
2022-03-31 13:08:01,167 - INFO - allennlp.common.params - factors.evaluation_factors = None
2022-03-31 13:08:01,169 - INFO - allennlp.commo

2022-03-31 13:08:01,809 - INFO - allennlp.common.params - id = roberta-sst
2022-03-31 13:08:01,810 - INFO - allennlp.common.params - registered_model_name = None
2022-03-31 13:08:01,814 - INFO - allennlp.common.params - model_class = None
2022-03-31 13:08:01,817 - INFO - allennlp.common.params - registered_predictor_name = None
2022-03-31 13:08:01,818 - INFO - allennlp.common.params - display_name = RoBERTa large
2022-03-31 13:08:01,822 - INFO - allennlp.common.params - task_id = sentiment-analysis
2022-03-31 13:08:01,838 - INFO - allennlp.common.params - model_usage.archive_file = stanford-sentiment-treebank-roberta.2021-03-11.tar.gz
2022-03-31 13:08:01,848 - INFO - allennlp.common.params - model_usage.training_config = classification/stanford_sentiment_treebank_roberta.jsonnet
2022-03-31 13:08:01,853 - INFO - allennlp.common.params - model_usage.install_instructions = pip install allennlp==2.4.0 allennlp-models==2.4.0
2022-03-31 13:08:01,871 - INFO - allennlp.common.params - model_us

...

2022-03-31 13:08:10,275 - INFO - allennlp.common.params - model_details.contact = allennlp-contact@allenai.org
2022-03-31 13:08:10,277 - INFO - allennlp.common.params - intended_use.primary_uses = None
2022-03-31 13:08:10,282 - INFO - allennlp.common.params - intended_use.primary_users = None
2022-03-31 13:08:10,283 - INFO - allennlp.common.params - intended_use.out_of_scope_use_cases = None
2022-03-31 13:08:10,286 - INFO - allennlp.common.params - factors.relevant_factors = None
2022-03-31 13:08:10,289 - INFO - allennlp.common.params - factors.evaluation_factors = None
2022-03-31 13:08:10,292 - INFO - allennlp.common.params - metrics.model_performance_measures = None
2022-03-31 13:08:10,296 - INFO - allennlp.common.params - metrics.decision_thresholds = None
2022-03-31 13:08:10,297 - INFO - allennlp.common.params - metrics.variation_approaches = None
2022-03-31 13:08:10,300 - INFO - allennlp.common.params - evaluation_data.dataset = None
2022-03-31 13:08:10,302 - INFO - allennlp.commo

2022-03-31 13:08:10,803 - INFO - allennlp.common.params - model_details.description = This `Model` implements a basic text classifier. The text is embedded into a text field using a RoBERTa-large model. The resulting sequence is pooled using a cls_pooler `Seq2VecEncoder` and then passed to a linear classification layer, which projects into the label space.
2022-03-31 13:08:10,804 - INFO - allennlp.common.params - model_details.short_description = RoBERTa finetuned on MNLI.
2022-03-31 13:08:10,805 - INFO - allennlp.common.params - model_details.developed_by = Liu et al
2022-03-31 13:08:10,812 - INFO - allennlp.common.params - model_details.contributed_by = Dirk Groeneveld
2022-03-31 13:08:10,813 - INFO - allennlp.common.params - model_details.date = 2020-07-29
2022-03-31 13:08:10,815 - INFO - allennlp.common.params - model_details.version = 1
2022-03-31 13:08:10,820 - INFO - allennlp.common.params - model_details.model_type = RoBERTa
2022-03-31 13:08:10,822 - INFO - allennlp.common.para

2022-03-31 13:08:11,240 - INFO - allennlp.common.params - evaluation_data.dataset.processed_url = https://s3-us-west-2.amazonaws.com/allennlp/datasets/squad/squad-dev-v2.0.json
2022-03-31 13:08:11,243 - INFO - allennlp.common.params - evaluation_data.dataset.url = https://rajpurkar.github.io/SQuAD-explorer/explore/2.0/dev/
2022-03-31 13:08:11,246 - INFO - allennlp.common.params - evaluation_data.motivation = None
2022-03-31 13:08:11,248 - INFO - allennlp.common.params - evaluation_data.preprocessing = None
2022-03-31 13:08:11,256 - INFO - allennlp.common.params - training_data.dataset.name = SQuAD training set
2022-03-31 13:08:11,258 - INFO - allennlp.common.params - training_data.dataset.processed_url = https://s3-us-west-2.amazonaws.com/allennlp/datasets/squad/squad-train-v2.0.json
2022-03-31 13:08:11,264 - INFO - allennlp.common.params - training_data.dataset.url = https://rajpurkar.github.io/SQuAD-explorer/explore/2.0/dev/
2022-03-31 13:08:11,268 - INFO - allennlp.common.params - t

2022-03-31 13:08:11,706 - INFO - allennlp.common.params - task_id = sentiment-analysis
2022-03-31 13:08:11,708 - INFO - allennlp.common.params - model_usage.archive_file = basic_stanford_sentiment_treebank-2020.06.09.tar.gz
2022-03-31 13:08:11,710 - INFO - allennlp.common.params - model_usage.training_config = classification/basic_stanford_sentiment_treebank.jsonnet
2022-03-31 13:08:11,711 - INFO - allennlp.common.params - model_usage.install_instructions = pip install allennlp==2.1.0 allennlp-models==2.1.0
2022-03-31 13:08:11,712 - INFO - allennlp.common.params - model_usage.overrides = None
2022-03-31 13:08:11,715 - INFO - allennlp.common.params - model_details.description = This model uses GloVe embeddings and is trained on the binary classification setting of the Stanford Sentiment Treebank. It achieves about 87% on the test set.
2022-03-31 13:08:11,716 - INFO - allennlp.common.params - model_details.short_description = LSTM binary classifier with GloVe embeddings.
2022-03-31 13:08

2022-03-31 13:08:13,468 - INFO - allennlp.common.params - model.text_field_embedder.token_embedders.tokens.sparse = False
2022-03-31 13:08:13,470 - INFO - allennlp.common.params - model.text_field_embedder.token_embedders.tokens.vocab_namespace = tokens
2022-03-31 13:08:13,473 - INFO - allennlp.common.params - model.text_field_embedder.token_embedders.tokens.pretrained_file = None
2022-03-31 13:08:13,584 - INFO - allennlp.common.params - model.encoder.type = alternating_lstm
2022-03-31 13:08:13,585 - INFO - allennlp.common.params - model.encoder.input_size = 200
2022-03-31 13:08:13,587 - INFO - allennlp.common.params - model.encoder.hidden_size = 300
2022-03-31 13:08:13,587 - INFO - allennlp.common.params - model.encoder.num_layers = 8
2022-03-31 13:08:13,589 - INFO - allennlp.common.params - model.encoder.recurrent_dropout_probability = 0.1
2022-03-31 13:08:13,596 - INFO - allennlp.common.params - model.encoder.use_highway = True
2022-03-31 13:08:13,604 - INFO - allennlp.common.params

## Create and Run Tests

In [80]:
#t.run(predict_and_conf)
#t.summary(format_example_fn=format_srl)
#result_dict = append_result_dict_by_test(result_dict, test1, 'MFT')

In [112]:
%run utils.ipynb

result_dict = create_result_dict()
editor = Editor()
nr_of_samples = 1

# loop through predictor
predict_and_conf = wrap_predict(predict_srl, srl_predictor) 

# generate, execute and store results of tests

'''
# test set 1
print('\n\n # Phenomenon 1 - Voice \n')
tests = generate_voice(editor, nr_of_samples)
result_dict = run_tests(result_dict, tests, 'MFT', predict_and_conf)


# test set 2
print('\n\n # Phenomenon 2 - Long Distance Dependencies \n')
tests = generate_long_distance_dependencies(editor, nr_of_samples)
result_dict = run_tests(result_dict, tests, 'MFT', predict_and_conf)


# test set 3
print('\n\n # Phenomenon 3 - Location vs Direction Confusion \n')
tests = generate_loc_vs_dir(editor, nr_of_samples)
result_dict = run_tests(result_dict, tests, 'MFT', predict_and_conf)


print('\n\n # Phenomenon 4 - Dative Alternation \n')
# test set 4
tests = generate_dative_alternation(editor, nr_of_samples)
result_dict = run_tests(result_dict, tests, 'MFT', predict_and_conf)
'''




#pp.pprint(result_dict)
result_dict

{'capabilities': {}}

## Evaluation

In [95]:
arg = 'R-ARGM-LOC'
#arg.startswith('ARG')
if 'ARG' in arg and not arg.startswith('ARG'):
    arg = arg[arg.find('-')+1:]
    
arg

'ARGM-LOC'